# Ensemble Learning

## Initial Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [6]:
# Create our features
X = pd.get_dummies(df.drop(columns='loan_status', axis=1), drop_first=True)

# Create our target
Y = df.loan_status

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.451066,0.177238,0.876121,0.616839,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.497603,0.381873,0.329446,0.486161,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
Y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    random_state=1,
    stratify=Y
)

X_train.shape

(51612, 86)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
standard_scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = standard_scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_train_scaler = X_scaler.transform(X_train)
X_test_scaler = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
balanced_classifier = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
balanced_classifier.fit(X_train, Y_train)

BalancedRandomForestClassifier(random_state=1)

In [14]:
# Calculated the balanced accuracy score
predicted_score = balanced_classifier.predict(X_test)
balanced_accuracy_score(Y_test, predicted_score)

0.795829959187949

In [15]:
# Display the confusion matrix
confusion_matrix(Y_test, predicted_score)

array([[   62,    25],
       [ 2071, 15047]], dtype=int64)

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(Y_test, predicted_score, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.0291    0.7126    0.8790    0.0559    0.7915    0.6160        87
   low_risk     0.9983    0.8790    0.7126    0.9349    0.7915    0.6368     17118

avg / total     0.9934    0.8782    0.7135    0.9304    0.7915    0.6367     17205



In [17]:
# List the features sorted in descending order by feature importance
features_descending = pd.DataFrame(
    balanced_classifier.feature_importances_,
    index=X_train.columns,
    columns=['features descending']
)

features_descending.sort_values('features descending', ascending=False)

,features descending
total_rec_prncp,0.078795
total_rec_int,0.068296
total_pymnt_inv,0.063323
total_pymnt,0.060703
last_pymnt_amnt,0.048325
...,...
num_tl_120dpd_2m,0.000000
collection_recovery_fee,0.000000
num_tl_30dpd,0.000000
delinq_amnt,0.000000


### Easy Ensemble Classifier

In [18]:
# Train the Classifier
ensemble_classifier = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ensemble_classifier.fit(X_train, Y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [19]:
# Calculated the balanced accuracy score
predicted_ensemble_score = ensemble_classifier.predict(X_test)
balanced_accuracy_score(Y_test, predicted_ensemble_score)

0.9263912558266958

In [20]:
# Display the confusion matrix
confusion_matrix(Y_test, predicted_ensemble_score)

array([[   79,     8],
       [  946, 16172]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(Y_test, predicted_ensemble_score, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.0771    0.9080    0.9447    0.1421    0.9262    0.8547        87
   low_risk     0.9995    0.9447    0.9080    0.9713    0.9262    0.8610     17118

avg / total     0.9948    0.9446    0.9082    0.9672    0.9262    0.8610     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The easy ensemble classifier had the best balanced accuracy score.

2. Which model had the best recall score?

    The easy ensemble classifier also had the best recall score.

3. Which model had the best geometric mean score?

    The easy ensemble had the best geometric mean score as well, scoring at 0.9262.

4. What are the top three features?

    The top three features are the below:
    * total_rec_prncp at 0.078795
    * total_rec_int at 0.068296
    * total_pymnt_inv at 0.063323